In [1]:
from langchain import load

In [2]:
from langchain_community.llms.ollama import Ollama
from langchain_community.embeddings.ollama import OllamaEmbeddings

In [3]:
from langchain_community.document_loaders import YoutubeLoader

In [4]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

In [5]:
#### INDEXING ####

# Load Documents
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

In [6]:
# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [7]:
llm = Ollama(model="phi")
# embed_model = HuggingFaceEmbeddings(model_name="nomic-embed-text")

In [31]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
embed_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


In [38]:
from langchain_nomic.embeddings import NomicEmbeddings
# from langchain_nomic import NomicEmbeddings
embed_nomic = NomicEmbeddings(model="nomic-embed-text")
embed_nomic

In [39]:
vectorstore_nomic = Chroma.from_documents(documents=splits, 
                                    embedding=embed_nomic)

ValueError: You have not configured your Nomic API token. Run `nomic login` to configure.

In [12]:
# Embed
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=embed_model)

In [13]:
# vectorstore = Chroma.from_documents(documents=splits, 
#                                     embedding=embed_model)

retriever = vectorstore.as_retriever()

In [15]:
#### RETRIEVAL and GENERATION ####

# Prompt
prompt = hub.pull("rlm/rag-prompt")

In [16]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [24]:
llm = Ollama(model="phi", temperature=0, timeout=300)

In [25]:
llm

Ollama(model='phi', temperature=0.0, timeout=300)

In [26]:
# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [27]:
# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [28]:
# Question
rag_chain.invoke("What is Task Decomposition?")

" Task decomposition refers to breaking down complex tasks into smaller and simpler steps. This technique is often used by artificial intelligence systems to enhance their performance on challenging tasks. One popular approach to task decomposition is the chain of thought (CoT) method, which involves prompting the model to think step-by-step to decompose a hard task into manageable components. Another approach is tree of thoughts (ToT), which explores multiple reasoning possibilities at each step and generates multiple thoughts per step, creating a tree structure. Task decomposition can be done using LLM with simple prompts, task-specific instructions, or human inputs. However, the limited context capacity of LLMs poses challenges in long-term planning and effective exploration of the solution space. The reflexion framework is one approach to address these challenges by incorporating self-reflection into the planning process. This involves showing two-shot examples to the system, where